In [ ]:
# questions I have as I think about this more:
# when we do the whole deduplication, we should only be doing that on TRAIN, right?

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
import pandas as pd
import numpy as np
import copy
import time
import pickle

pd.set_option('max_colwidth', 100)

In [2]:
splits = "splits_1571429760//"

In [3]:
data_folder = "data//"
train_path = data_folder + splits + "train.csv"
dev_path = data_folder + splits + "dev.csv"

In [4]:
this_model_folder = "unigram_l1//"

In [5]:
models_folder = "models//"
pkl_filename = models_folder + this_model_folder + "model.pkl"
vectorizer_filename = models_folder + this_model_folder + "vectorizer.pkl"

In [6]:
train_df = pd.read_csv(train_path, index_col=0)

/home/emilyrapport/anaconda3/envs/w266-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
vectorizer = HashingVectorizer()

In [13]:
responses = train_df.response_text
xs = []

# we have to do this in chunks because fit_transform
# can't handle the whole thing in memory in string form
for i in range(1,11):
    print("starting chunk {}".format(i))
    chunk = responses.iloc[(i-1)*1000000:i*1000000]
    X = vectorizer.fit_transform(chunk.apply(lambda x: np.str_(x)))
    xs.append(X)

starting chunk 1
starting chunk 2
starting chunk 3
starting chunk 4
starting chunk 5
starting chunk 6
starting chunk 7
starting chunk 8
starting chunk 9
starting chunk 10


In [14]:
# this is how we recombine the chunks 
X = vstack(xs)

In [15]:
X.shape

(9879016, 1048576)

In [16]:
Y = train_df.op_gender.values

In [12]:
model = LogisticRegression(random_state=0, solver='saga', penalty='l1')

In [ ]:
time_start = time.time()


model.fit(X,Y)
currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Model trained in {}".format(timeStr))

In [ ]:
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [ ]:
with open(vectorizer_filename, 'wb') as file:
    pickle.dump(vectorizer, file)

In [ ]:
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)
with open(vectorizer_filename, 'rb') as file:
    vectorizer = pickle.load(file)

In [ ]:
dev_df = pd.read_csv(dev_path, index_col=0)

In [ ]:
dev_x = vectorizer.fit_transform(dev_df.response_text.apply(lambda x: np.str_(x)))

In [ ]:
dev_y = dev_df.op_gender.values

In [ ]:
model.score(dev_x, dev_y)

In [ ]:
preds = model.predict(dev_x)

In [ ]:
probs = model.predict_proba(dev_x)

In [ ]:
probs.shape

In [ ]:
dev_df['preds'] = preds

In [ ]:
prob_m = [prob[0] for prob in probs]
prob_f = [prob[1] for prob in probs]

In [ ]:
dev_df['prob_m'] = prob_m
dev_df['prob_f'] = prob_f

In [ ]:
dev_df.head()

In [ ]:
#### get all wrong predictions
wrong_preds = dev_df[dev_df['op_gender']!=dev_df['preds']]

In [ ]:
wrong_preds.op_gender.value_counts()

In [ ]:
# predicting W somewhat more often than the first model did 
dev_df.preds.value_counts()['W'] / dev_df.preds.value_counts()['M']

In [ ]:
label_m_pred_f = wrong_preds[wrong_preds.op_gender=='M']
label_f_pred_m = wrong_preds[wrong_preds.op_gender=='W']

In [ ]:
label_m_pred_f.sort_values('prob_f', ascending=False)[:1000]